In [ ]:
#XGBOOST 15 YEAR NO SENTIMENT
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt


df_btc = pd.read_csv('/notebooks/BITCOIN_DATA.csv', parse_dates=['datetime'])
df_btc.sort_values('datetime', inplace=True)
df_btc = df_btc[['datetime', 'close','volume']]
print(df_btc.tail())

# Define features and target
features = ['close', 'volume']
target = 'close'

# Split data based on datetime
train_data = df_btc[(df_btc['datetime'] >= '2020-01-01') & (df_btc['datetime'] < '2024-01-01')].copy()
test_data = df_btc[df_btc['datetime'] >= '2024-01-01'].copy()

X_train_full = train_data[features].values
y_train_full = train_data[target].values
X_test = test_data[features].values
y_test = test_data[target].values

# ----------------------------
# 2. Time Series Cross-Validation
# ----------------------------
tscv = TimeSeriesSplit(n_splits=5)
fold = 1
cv_scores = []

for train_index, val_index in tscv.split(X_train_full):
    # Split into train and validation sets for this fold
    X_fold_train, X_fold_val = X_train_full[train_index], X_train_full[val_index]
    y_fold_train, y_fold_val = y_train_full[train_index], y_train_full[val_index]
    
    # Build and train the model for this fold
    xgb_fold_model = XGBRegressor(
        objective='reg:squarederror', 
        n_estimators=200, 
        learning_rate=0.1, 
        random_state=42
    )
    xgb_fold_model.fit(X_fold_train, y_fold_train)
    
    # Predict on the validation set
    val_preds = xgb_fold_model.predict(X_fold_val)
    
    # Compute R² for this fold
    fold_r2 = r2_score(y_fold_val, val_preds)
    cv_scores.append(fold_r2)
    
    print(f"Fold {fold} R2 Score: {fold_r2:.4f}")
    fold += 1

print(f"\nTimeSeriesSplit Mean R2 Score: {np.mean(cv_scores):.4f}")

# ----------------------------
# 3. Final Model Training and Testing
# ----------------------------
# After evaluating via TSCV, retrain on the entire training data
final_model = XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=200, 
    learning_rate=0.1, 
    random_state=42
)
final_model.fit(X_train_full, y_train_full)

# Predict on the test set
test_preds = final_model.predict(X_test)

# ----------------------------
# 4. Performance Evaluation on Test Set
# ----------------------------
mse = mean_squared_error(y_test, test_preds)
mae = mean_absolute_error(y_test, test_preds)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, test_preds) * 100
r2 = r2_score(y_test, test_preds)

print("\nFinal Test Set Performance:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R2 Score: {r2:.2f}")

# ----------------------------
# 5. Visualization
# ----------------------------
plt.figure(figsize=(12, 6))
plt.plot(test_data['datetime'], y_test, label='Actual Close Price', color='blue')
plt.plot(test_data['datetime'], test_preds, label='Predicted Close Price', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction: Actual vs Predicted (XGBoost)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
